In [1]:
from py2neo import Graph
import json
import requests

## Import the NPM registry into Neo4j

In [2]:
REGISTRY_URL = "https://skimdb.npmjs.com/registry/"

In [40]:
graph = Graph()

In [4]:
def loadPackage(package_name):
    r = requests.get(REGISTRY_URL + package_name)
    d = r.json()
    latest_tag = d['dist-tags']['latest']
    current_version = d['versions'][latest_tag]
    print(latest_tag)
    graph.cypher.execute(PACKAGE_IMPORT_QUERY, parameters={'json': d, 'current_version': current_version})
    #return d, current_version

In [53]:
PACKAGE_IMPORT_QUERY = '''
WITH {json} AS data 
MERGE (p:Package {package_id: data._id})
SET p._rev = data._rev,
    p.name = data.name,
    p.description = data.description

MERGE (author:Developer {email: coalesce(data.author.email, "N/A")})
SET author.name = data.author.name
CREATE UNIQUE (p)<-[:AUTHOR_OF]-(author)

MERGE (rep:Repository {url: coalesce(data.repository.url, "N/A")})
SET rep.type = data.repository.type
CREATE UNIQUE (p)-[:HAS_REPO]->(rep)

FOREACH (maint IN data.maintainers | 
    MERGE (m:Developer {email: coalesce(maint.email, "N/A")})
    SET m.name = maint.name
    CREATE UNIQUE (m)-[:MAINTAINS]->(p)
)

FOREACH (cont IN data.contributors |
    MERGE (c:Developer {email: coalesce(cont.email, "N/A")})
    SET c.name = cont.name
    CREATE UNIQUE (c)-[:CONTRIBUTES_TO]->(p)
)

FOREACH (kw IN data.keywords |
    MERGE (k:Keyword {word: kw})
    CREATE UNIQUE (k)<-[:DEALS_WITH]-(p)
)

//WITH data, p, {current_version} AS current_version
WITH data,p
UNWIND keys(data.versions) AS cv

MERGE (v:Version {version: data.versions[cv]["version"]})<-[:HAS_VERSION]-(p)
//CREATE UNIQUE (p)-[:HAS_VERISON]->(v)

MERGE (l:License {name: coalesce(data.versions[cv]["license"], "N/A")})
CREATE UNIQUE (v)-[:LICENSED_UNDER]->(l)

FOREACH (dep IN keys(data.versions[cv]["dependencies"]) |
    MERGE (dep_p:Package {package_id: dep})
    MERGE (dep_v:Version {version: replace(replace(data.versions[cv]["dependencies"][dep], "~", ""), ">=", "")})<-[:HAS_VERSION]-(dep_p)
    //MERGE (dep_v)<-[:HAS_VERSION]-(dep_p)
    //SET dep_p.version = replace(replace(current_version.dependencies[dep], "~", ""), ">=", "")
    CREATE UNIQUE (dep_v)<-[:DEPENDS_ON]-(v)
)



// What is the users array??
//FOREACH (user IN keys(data.users) |
//    MERGE (user:Package {package_id: })
//    CREATE UNIQUE (p)<-[:USES]-()

'''

GET_PENDING_PACKAGE_QUERY = '''
MATCH (p:Package) WHERE NOT has(p.name)
RETURN p.package_id AS package_id LIMIT 1
'''

In [49]:
loadPackage("express")

4.13.4


In [134]:
graph.cypher.execute(PACKAGE_IMPORT_QUERY, parameters={'json': package_json, 'current_version': latest_tag})

In [86]:
express.keys()

dict_keys(['users', 'maintainers', 'homepage', 'readme', 'readmeFilename', 'dist-tags', 'versions', 'description', 'license', '_rev', 'repository', 'name', 'bugs', '_id', 'time', 'keywords', 'author'])

In [54]:
result = graph.cypher.execute(GET_PENDING_PACKAGE_QUERY)
while result:
    new_package = result.one
    print(new_package)
    loadPackage(new_package)
    result = graph.cypher.execute(GET_PENDING_PACKAGE_QUERY)

inherits
2.0.1


InvalidType: Property values can only be of primitive types or arrays thereof

In [151]:
loadPackage("cookie-signature")

1.0.6
